In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
import kagglehub
import matplotlib.pyplot as plt

class LeapGestureDataset(Dataset):
    def __init__(self, num_images=None, color_mode='L'):
        """
        Loads all gesture classes from the LeapGesture dataset.

        Args:
            num_images (int): optional limit for debugging (load up to N images total)
            color_mode (str): 'L' for grayscale (default) or 'RGB' for 3-channel input
        """
        # Download dataset from Kaggle
        self.path = kagglehub.dataset_download('gti-upm/leapgestrecog')
        self.base_dir = os.path.join(self.path, 'leapGestRecog')

        self.images = []
        self.labels = []

        gesture_label_map = {}  # map gesture folder names to numeric labels
        label_counter = 0

        for person_folder in sorted(os.listdir(self.base_dir)):
            person_path = os.path.join(self.base_dir, person_folder)
            if not os.path.isdir(person_path):
                continue

            for gesture_folder in sorted(os.listdir(person_path)):
                gesture_path = os.path.join(person_path, gesture_folder)
                if not os.path.isdir(gesture_path):
                    continue

                # assign numeric label
                if gesture_folder not in gesture_label_map:
                    gesture_label_map[gesture_folder] = label_counter
                    label_counter += 1

                label = gesture_label_map[gesture_folder]

                for filename in os.listdir(gesture_path):
                    if not filename.endswith('.png'):
                        continue

                    img_path = os.path.join(gesture_path, filename)
                    img = Image.open(img_path).convert(color_mode)
                    img = np.asarray(img) / 255.0
                    self.images.append(img)
                    self.labels.append(label)

                    if num_images and len(self.images) >= num_images:
                        break

                if num_images and len(self.images) >= num_images:
                    break
            if num_images and len(self.images) >= num_images:
                break

        self.label_map = gesture_label_map
        print(f"Loaded {len(self.images)} total images across {len(gesture_label_map)} gestures.")
        print("Gesture label mapping:", gesture_label_map)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]

        # Convert to tensor
        if img.ndim == 2:  # grayscale
            img_tensor = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # (1, H, W)
        else:
            img_tensor = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # (3, H, W)

        return img_tensor, label


In [ ]:
dataset = LeapGestureDataset(num_images=500, color_mode='L')

In [ ]:
print(len(dataset))
img, label = dataset[0]
print(img.shape, label)

# Display image
plt.imshow(img.squeeze(), cmap='gray')
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()

In [ ]:
dataset.path